# Sample notebook for usage

This notebook shows a sample flow for running portfolio simluations.

In [1]:
from investment_simulator.simulator import InvestmentSimulator
from investment_simulator.data_retriever import DataRetriever
from investment_simulator import set_objective
from investment_simulator import set_constraints

In [2]:
# Set a list of tickers for analysis
tickers = [
    "GLD", # SPDR Gold Trust
    "DBC", # Invesco DB Commodity Index Tracking Fund 5%
    "IVV", # iShares Core S&P 500 ETF 10%
    "ACWI", # iShares MSCI ACWI ETF 25%
    "JPST", # JPMorgan Ultra-Short Income ETF 25%
]

In [ ]:
import os

os.environ["ALPHA_VANTAGE_API_KEY"] = ""  # TODO: add your own API key

In [ ]:
# Optional: load latest snapshot of data from Alpha Vantage
# Prior to running this, set the environment variable ALPHA_VANTAGE_API_KEY to your Alpha Vantage API key.
data_retriever = DataRetriever()

# first, get CPI and inflation data
data_retriever.get_cpi_data()
data_retriever.get_inflation_data()

# then, get ETF / stock price data
for ticker in tickers:
    data_retriever.get_price_data(ticker)

In [ ]:
sim = InvestmentSimulator(
    tickers=tickers,
    input_date="2017-01-01",
    output_date="2025-01-01",
)

# Set the objective and constraints
set_objective.maximize_ema_return(sim.data, sim.tickers, smoothing_window=100)
set_constraints.keep_long_positions_only()
set_constraints.cap_maximal_dod_loss_at_threshold(sim.data, sim.tickers, 0.1)
set_constraints.cap_maximal_drawdown_at_threshold(sim.data, sim.tickers, 0.2)

# Run optimization
sim.optimize()

# Generate report
sim.generate_report()

# Clear the optimizer
sim.clear()

# Examine the report
sim.report